In [1]:
import pandas as pd

In [2]:
full_df = pd.read_csv("../raw_data/Bus Arrival Departure Times Jan-Mar 2020.csv",
                 parse_dates=["scheduled", "actual", "service_date"])
full_df.head()

,service_date,route_id,direction,half_trip_id,stop_id,time_point_id,time_point_order,point_type,standard_type,scheduled,actual,earliness,scheduled_headway,headway
0,2020-01-01,01,Inbound,46202828.0,110,hhgat,1,Startpoint,Schedule,1900-01-01 06:10:00,1900-01-01 06:11:58,676.0,NaN,NaN
1,2020-01-01,01,Inbound,46202828.0,67,maput,2,Midpoint,Schedule,1900-01-01 06:15:00,1900-01-01 06:15:48,-37.0,NaN,NaN
2,2020-01-01,01,Inbound,46202828.0,72,cntsq,3,Midpoint,Schedule,1900-01-01 06:18:00,1900-01-01 06:18:52,-10.0,NaN,NaN
3,2020-01-01,01,Inbound,46202828.0,75,mit,4,Midpoint,Schedule,1900-01-01 06:21:00,1900-01-01 06:22:07,-34.0,NaN,NaN
4,2020-01-01,01,Inbound,46202828.0,79,hynes,5,Midpoint,Schedule,1900-01-01 06:25:00,1900-01-01 06:25:54,19.0,NaN,NaN


In [3]:
# inbound trips of the #1 bus
one_inbound = full_df.query("route_id=='01' and direction=='Inbound'")
one_inbound = one_inbound.dropna(subset=["half_trip_id"])
one_trips = one_inbound.pivot(index="half_trip_id", columns="time_point_id", values="actual")

#reorder cols
time_point_ids = ["hhgat", "maput", "cntsq", "mit", "hynes", "masta", "Wasma", "Melwa", "Dudly"]
one_trips = one_trips[time_point_ids]

print(len(one_trips))
one_trips.head()

9891


time_point_id,hhgat,maput,cntsq,mit,hynes,masta,Wasma,Melwa,Dudly
half_trip_id,,,,,,,,,
46202752.0,1900-01-01 06:49:49,1900-01-01 06:53:22,1900-01-01 06:58:01,1900-01-01 07:01:52,1900-01-01 07:05:57,1900-01-01 07:12:25,1900-01-01 07:14:42,1900-01-01 07:20:57,1900-01-01 07:21:42
46202754.0,1900-01-01 07:49:34,1900-01-01 07:53:04,1900-01-01 07:55:01,1900-01-01 07:57:18,1900-01-01 07:59:36,1900-01-01 08:03:12,1900-01-01 08:06:34,1900-01-01 08:10:23,1900-01-01 08:11:08
46202756.0,1900-01-01 08:57:33,1900-01-01 09:01:42,1900-01-01 09:04:40,1900-01-01 09:09:07,1900-01-01 09:12:35,1900-01-01 09:14:51,1900-01-01 09:18:13,1900-01-01 09:23:52,1900-01-01 09:24:38
46202758.0,1900-01-01 10:19:20,1900-01-01 10:22:57,1900-01-01 10:26:10,1900-01-01 10:29:26,1900-01-01 10:33:48,1900-01-01 10:38:45,1900-01-01 10:44:08,1900-01-01 10:49:34,1900-01-01 10:50:22
46202760.0,1900-01-01 11:54:27,1900-01-01 11:59:07,1900-01-01 12:02:49,1900-01-01 12:07:55,1900-01-01 12:13:00,1900-01-01 12:17:51,1900-01-01 12:21:24,1900-01-01 12:26:08,1900-01-01 12:26:42


In [4]:
# Data cleaning
one_trips.dropna(inplace = True)
one_trips.query("hhgat < maput and Melwa < Dudly", inplace=True)
len(one_trips)

9078

In [5]:
hhgat = one_inbound[one_inbound.time_point_id=="hhgat"]
hhgat.set_index("half_trip_id", inplace=True, verify_integrity=True)

one_trips.insert(0, "service_date", hhgat.service_date)
one_trips.insert(1, "duration", (one_trips["Dudly"] - one_trips["hhgat"]).dt.seconds)
one_trips.head()

time_point_id,service_date,duration,hhgat,maput,cntsq,mit,hynes,masta,Wasma,Melwa,Dudly
half_trip_id,,,,,,,,,,,
46202752.0,2020-01-01,1913,1900-01-01 06:49:49,1900-01-01 06:53:22,1900-01-01 06:58:01,1900-01-01 07:01:52,1900-01-01 07:05:57,1900-01-01 07:12:25,1900-01-01 07:14:42,1900-01-01 07:20:57,1900-01-01 07:21:42
46202754.0,2020-01-01,1294,1900-01-01 07:49:34,1900-01-01 07:53:04,1900-01-01 07:55:01,1900-01-01 07:57:18,1900-01-01 07:59:36,1900-01-01 08:03:12,1900-01-01 08:06:34,1900-01-01 08:10:23,1900-01-01 08:11:08
46202756.0,2020-01-01,1625,1900-01-01 08:57:33,1900-01-01 09:01:42,1900-01-01 09:04:40,1900-01-01 09:09:07,1900-01-01 09:12:35,1900-01-01 09:14:51,1900-01-01 09:18:13,1900-01-01 09:23:52,1900-01-01 09:24:38
46202758.0,2020-01-01,1862,1900-01-01 10:19:20,1900-01-01 10:22:57,1900-01-01 10:26:10,1900-01-01 10:29:26,1900-01-01 10:33:48,1900-01-01 10:38:45,1900-01-01 10:44:08,1900-01-01 10:49:34,1900-01-01 10:50:22
46202760.0,2020-01-01,1935,1900-01-01 11:54:27,1900-01-01 11:59:07,1900-01-01 12:02:49,1900-01-01 12:07:55,1900-01-01 12:13:00,1900-01-01 12:17:51,1900-01-01 12:21:24,1900-01-01 12:26:08,1900-01-01 12:26:42


In [6]:
one_trips.to_csv("../data/one_trips.csv")